In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 23962978
paper_name = 'neumuller_perrimon_2013' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/NIHMS559532-supplement-Table_S1.xls', sheet_name='SGA screening results')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4209 x 6


In [7]:
original_data.head()

,ORF,GeneID,Symbol,Distance (D),Fragmentation score (F),Comment
0,YMR235C,855275,RNA1,-5.112764,1.828640,selected as hit
1,YLR440C,851161,SEC39,-4.954860,1.466702,selected as hit
2,YPR181C,856311,SEC23,-4.922468,1.266921,selected as hit
3,YGL145W,852732,TIP20,-4.757409,1.538039,selected as hit
4,YPL169C,855934,MEX67,-4.705325,2.037921,selected as hit


In [8]:
original_data['orf'] = original_data['ORF'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, GeneID, Symbol, Distance (D), Fragmentation score (F), Comment, orf]
Index: []


In [12]:
original_data['data'] = pd.to_numeric(original_data['Distance (D)'], errors='coerce')

In [13]:
original_data.set_index('orf', inplace=True)

In [14]:
original_data = original_data[['data']].copy()

In [15]:
original_data = original_data.groupby(original_data.index).mean()

In [16]:
original_data.shape

(4209, 1)

# Prepare the final dataset

In [17]:
data = original_data.copy()

In [18]:
dataset_ids = [21849]
datasets = datasets.reindex(index=dataset_ids)

In [19]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [20]:
data.head()

dataset_id,21849
data_type,value
orf,
YAL001C,0.617998
YAL002W,0.254927
YAL005C,0.182785
YAL007C,0.164117
YAL008W,0.080019


## Subset to the genes currently in SGD

In [21]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 15


In [22]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,21849
,data_type,value
gene_id,orf,
1,YAL001C,0.617998
2,YAL002W,0.254927
4,YAL005C,0.182785
5,YAL007C,0.164117
6,YAL008W,0.080019


# Normalize

In [23]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [24]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [25]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id          21849          
data_type           value    valuez
gene_id orf                        
1       YAL001C  0.617998  1.006568
2       YAL002W  0.254927  0.461381
4       YAL005C  0.182785  0.353054
5       YAL007C  0.164117  0.325021
6       YAL008W  0.080019  0.198739

# Print out

In [26]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [27]:
from IO.save_data_to_db3 import *

In [28]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 23962978...
Inserting the new data...


100%|██████████| 1/1 [00:08<00:00,  8.25s/it]

Updating the data_modified_on field...
